# インシデント調査

M先輩から「このサーバがちょっと怪しいくって調査したいから、[インシデントレスポンス第3版](https://www.amazon.co.jp/dp/4822279871/)のp179に載ってるサーバの調査コマンド一式の結果のレポート作って。あ、あと、ポートが1つ LISTEN になってて http サーバになってるはずなんで、それが動いてるかも確認しといて。」と指示があった。

前回作業を踏まえて今回も作業するが、今後も同様の作業がありそうなので、リファクタリングしておく。

M先輩から「サービス止めないでファイルシステム舐めたりしてるので、作業が与えるサーバの負荷は気になる」とコメントがあったので、適時ロードアベレージを取っておくことにする。（コードは別のとこで作ったものを持ち込んだ）

システムの負荷状態を記録しておく。

In [ ]:
!pip install parse

In [ ]:
import datetime
JST = datetime.timezone(datetime.timedelta(hours=+9), 'JST')

import parse

uptime_list = []

uptime_str = !uptime
uptime_clumn = parse.parse("{}load average: {:g}, {:g}, {:g}", uptime_str[0])
uptime_clumn = list(uptime_clumn)[1:]
uptime_clumn.insert(0, datetime.datetime.now(JST))
uptime_list.append(uptime_clumn)
print(uptime_list)

# ライブレスポンス実施

## 事前準備

[作業対象のコンテナ](https://hub.docker.com/r/mnagaku/lc-minimal-notebook/)では、環境変数でTZ変えれるので、JSTに設定する。

In [ ]:
%env TZ JST-9

コマンドの追加インストール。

In [ ]:
!sudo apt-get update
!sudo apt-get upgrade -y
!sudo apt-get install -y sysv-rc-conf kmod lsof

Pythonのライブラリも更新しておく。

In [ ]:
!pip install -U pip
!pip install pip-review
!sudo -H pip-review --auto

追加したコマンドの確認。

In [ ]:
!which sysv-rc-conf
!which lsmod
!which lsof

## システムの日付と時間

In [ ]:
!date

## インストールされているソフトウェア

In [ ]:
!dpkg --get-selections

## ファイルシステム情報

In [ ]:
!mount -l

In [ ]:
!df -Th

In [ ]:
!fdisk -l

## OSのバージョン

In [ ]:
!cat /etc/issue

## カーネルのバージョン

In [ ]:
!uname -a

## 稼働時間

In [ ]:
!w

## cron

/var/spool/cron が存在しない前提で確認だけしておく。

In [ ]:
import sys
dirs = !sudo ls /var/spool
for s in filter(lambda x:x == "cron", dirs):
    sys.exit(-1)


## サービス

本には具体的なコマンドの指定がないが、ubuntuなので、sysv-rc-confを使う。[参考情報](https://www.laddy.info/2017/11/28793/)

In [ ]:
!sudo sysv-rc-conf --list

## ユーザアカウント

In [ ]:
!cat /etc/passwd

In [ ]:
!sudo cat /etc/shadow

## グループ

In [ ]:
!cat /etc/group

## ネットワークインタフェース

今なら、ipコマンドだよな。

In [ ]:
!ip a

In [ ]:
!ip -s l

## ルーティングテーブル

In [ ]:
!ip r

## ARPテーブル

In [ ]:
!ip n

## ネットワーク接続

TCP、UDPそれぞれssコマンドで見ることにする。

In [ ]:
!ss -nat

In [ ]:
!ss -nau

## ロードされているドライバ

In [ ]:
!lsmod

## オープン中のファイルとハンドル

In [ ]:
!lsof

## 実行中のプロセスとスレッド

In [ ]:
!ps auxwwwem

## 設定情報の保全

ファイルに日時のプリフィクスを付ける。

In [77]:
prefix = datetime.datetime.now(JST).strftime("%Y%m%d%H%M")
print(prefix)

201806011649


In [78]:
!sudo tar cvzf {prefix}_host_etc.tar.gz /etc/
!ls -la {prefix}_host_etc.tar.gz

tar: Removing leading `/' from member names
/etc/
/etc/passwd
/etc/rc1.d/
/etc/rc1.d/S02single
/etc/rc1.d/README
/etc/rc1.d/S01killprocs
/etc/cron.weekly/
/etc/cron.weekly/fstrim
/etc/selinux/
/etc/selinux/semanage.conf
/etc/rc.local
/etc/update-motd.d/
/etc/update-motd.d/00-header
/etc/update-motd.d/10-help-text
/etc/dhcp/
/etc/dhcp/dhclient-exit-hooks.d/
/etc/dhcp/dhclient-exit-hooks.d/timesyncd
/etc/rc6.d/
/etc/rc6.d/K03umountfs
/etc/rc6.d/K02umountnfs.sh
/etc/rc6.d/K05reboot
/etc/rc6.d/K01sendsigs
/etc/rc6.d/README
/etc/rc6.d/K01hwclock.sh
/etc/rc6.d/K01urandom
/etc/rc6.d/K04umountroot
/etc/sysctl.d/
/etc/sysctl.d/10-console-messages.conf
/etc/sysctl.d/10-link-restrictions.conf
/etc/sysctl.d/10-network-security.conf
/etc/sysctl.d/10-magic-sysrq.conf
/etc/sysctl.d/10-ipv6-privacy.conf
/etc/sysctl.d/10-ptrace.conf
/etc/sysctl.d/README
/etc/sysctl.d/10-zeropage.conf
/etc/sysctl.d/99-sysctl.conf
/etc/sysctl.d/10-kernel-hardening.conf
/etc/init.d/
/etc/init.d/checkroot-bootclean.sh
/etc

-rw-r--r-- 1 root users 262641 Jun  1 16:50 201806011649_host_etc.tar.gz


## システムログの保全

In [79]:
!sudo tar cvzf {prefix}_host_log.tar.gz /var/log/
!ls -la {prefix}_host_log.tar.gz

tar: Removing leading `/' from member names
/var/log/
/var/log/dmesg
/var/log/bootstrap.log
/var/log/fsck/
/var/log/fsck/checkroot
/var/log/fsck/checkfs
/var/log/faillog
/var/log/dpkg.log
/var/log/lastlog
/var/log/apt/
/var/log/apt/history.log
/var/log/apt/term.log
/var/log/alternatives.log
/var/log/btmp
/var/log/wtmp
-rw-r--r-- 1 root users 25598 Jun  1 16:51 201806011649_host_log.tar.gz


## ファイル一覧

In [80]:
!sudo find / -xdev -printf "%m;%Ax;%AT;%Tx;%TT;%Cx;%CT;%U;%G;%s;%p;\n" > {prefix}_host_files.txt
!ls -la {prefix}_host_files.txt

-rw-r--r-- 1 jovyan users 13832219 Jun  1 16:52 201806011649_host_files.txt


## ライブレスポンス出力チェックサム計算

ファイルを、host_etc.tar.gz、host_files.txt、host_log.tar.gzと3つ作ったので、md5sumしておく。

In [83]:
!md5sum {prefix}_host_etc.tar.gz {prefix}_host_files.txt {prefix}_host_log.tar.gz

bb368365ca62fa03444bb5c6631be76a  201806011649_host_etc.tar.gz
f94af1e3b3cd5a020cb81cb64a316cf9  201806011649_host_files.txt
eeee71f63443a100b0829b530373d6a2  201806011649_host_log.tar.gz


システムの負荷状態を記録しておく。

In [84]:
uptime_str = !uptime
uptime_clumn = parse.parse("{}load average: {:g}, {:g}, {:g}", uptime_str[0])
uptime_clumn = list(uptime_clumn)[1:]
uptime_clumn.insert(0, datetime.datetime.now(JST))
uptime_list.append(uptime_clumn)
print(uptime_list)

[[datetime.datetime(2018, 6, 1, 9, 28, 15, 192446, tzinfo=datetime.timezone(datetime.timedelta(0, 32400), 'JST')), 0.06, 0.02, 0.0], [datetime.datetime(2018, 6, 1, 16, 56, 25, 879132, tzinfo=datetime.timezone(datetime.timedelta(0, 32400), 'JST')), 0.03, 0.04, 0.0]]


## [手動]ユーザのシェル履歴

In [85]:
sys.exit(1)

SystemExit: 1

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


ターミナルで取ったhistoryを貼っておく。

取って来たのを貼っておく。
```
jovyan@8f91f8204652:~$ history
    1  history
    2  sudo -i
    3  exit
    4  history
    5  ls
    6  history
    7  sudo -i
    8  exit
    9  history
jovyan@8f91f8204652:~$ sudo -i
root@8f91f8204652:~# history
    1  history
    2  exit
    3  history
    4  ls
    5  exit
    6  history
root@8f91f8204652:~# exit
logout
jovyan@8f91f8204652:~$ exit
exit
```

# httpサーバの動確

LISTENポートを探して、httpでリクエスト出して、振る舞いを確認しておく。

まず、LISTENポートについては、[ネットワーク接続](#ネットワーク接続)で8888だけ全てのネットワークインタフェースに対してLISTENしていることが分かっている。外部に対して繋がっているネットワークインタフェースは[ルーティングテーブル](#ルーティングテーブル)と[ネットワークインタフェース](#ネットワークインタフェース)からeth0の172.17.0.2だと分かるので、ここにwgetしてみる。

In [ ]:
!wget -O - http://172.17.0.2:8888

302Foundで2回飛ばされて、ログインページに行って、htmlをもらえた。動いているのを確認できた。

# おしまい